<a href="https://colab.research.google.com/github/adrian-alejandro/BDMA/blob/main/proyecto/visualization/Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data sources:
*   [Gender Statistics Database](https://eige.europa.eu/gender-statistics/dgs)

### Parliament Representation:
*   [Seats held by women in national parliaments and governments](https://ec.europa.eu/eurostat/databrowser/view/sdg_05_50/default/table?lang=en)
*   [G1a. The proportion of women in the single/lower houses of the national/federal Parliaments of the Member States](https://eige.europa.eu/gender-statistics/dgs/indicator/bpfa_g_offic_g1__wmid_natparl)
*   [G1b. The proportion of women in the European Parliament](https://eige.europa.eu/gender-statistics/dgs/indicator/bpfa_g_offic_g1__wmid_eurparl)
*   [Parliamentary committees chaired by women and men](https://eige.europa.eu/gender-statistics/dgs/indicator/genmain_eqpol_gsp_area2__gsp_parlcomm)

### Violence:
*   [Physical and sexual violence to women by age group (2012 data)](https://ec.europa.eu/eurostat/databrowser/view/sdg_05_10/default/table?lang=en)
*   [Women who have experienced sexual violence during childhood, by age group](https://ec.europa.eu/eurostat/databrowser/view/gbv_ch_age/default/table?lang=en)
*   [Women who have experienced violence by any perpetrator, by age group](https://ec.europa.eu/eurostat/databrowser/view/gbv_any_age/default/table?lang=en)

### Other:
*   [Positions held by women in senior management positions](https://ec.europa.eu/eurostat/databrowser/view/sdg_05_60/default/table?lang=en)



In [2]:
!pip install -U "altair[all]" vega_datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [3]:
import pandas as pd
import altair as alt

# Asegúrate de que el archivo Excel esté disponible en el entorno de Colab
file_path = '/content/sdg_05_50_spreadsheet.xlsx'  # Actualiza con la ruta correcta a tu archivo

try:
    # Cargar los datos desde el archivo Excel, con encabezados en la fila correcta
    df = pd.read_excel(
        file_path,
        sheet_name='Sheet 1', # National Parliament
        header=9  # Los encabezados de años y regiones están en la fila 10
    )
except Exception as e:
    print("Error al cargar el archivo Excel:", e)

# Renombrar la primera columna para usarla como identificador de la región
df.rename(columns={df.columns[0]: 'Region'}, inplace=True)

# Descartamos las columnas que están de más
df = df.drop(columns=[x for x in df.columns if ':' in x])

# Descartamos las filas con nulos (antes hemos verificado que no haya nulos en filas con data)
df = df[~(df.isnull().sum(axis = 1) > 0)].reset_index(drop=True)

# Seleccionamos la fila correspondiente al overall EU, así lo utilizamos luego
EU = df.iloc[:2,0]

# Seleccionar solo las columnas de años que contienen datos, excluyendo las marcas 'd'
year_columns = df.columns[1:].tolist()

# Transformar los datos para análisis
df = df.melt(id_vars=['Region'], value_vars=year_columns, var_name='Año', value_name='Porcentaje')

# Convertir la columna 'Año' a numérico
df['Año'] = pd.to_numeric(df['Año'], errors='coerce')

# Convertir la columna 'Porcentaje' a numérico, manejando los valores no disponibles ':'
df['Porcentaje'] = pd.to_numeric(df['Porcentaje'].replace(':', None), errors='coerce')

# Agregamos la columna 'Grupo', que distingue si es EU o Europa (no-EU)
EU_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
                'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
                'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
                'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
                'Slovenia', 'Spain', 'Sweden']
df['Grupo'] = df.apply(lambda x: 'EU' if (x['Region'] in EU_countries) or ('European Union' in x['Region'])
                                      else 'Europa (no-EU)', 1)
df['Entidad'] = df.apply(lambda x: x['Grupo'] if 'European Union' in x['Region'] else x['Region'], 1)

# Crear una selección interactiva
selection = alt.selection_single(fields=['Region'], bind='legend')

# Gráfico interactivo de puntos conectados para todos los datos
chart = alt.Chart(df).mark_circle(size=60).encode(
    x='Año:N',
    y='Porcentaje:Q',
    color=alt.Color('Region:N', legend=alt.Legend(orient='right', columns=2, symbolLimit=50)),
    tooltip=['Region:N', 'Año:N', 'Porcentaje:Q']
).transform_filter(
    selection
).add_selection(
    selection
).properties(
    width=800,
    height=600
)

chart.display()

Error al cargar el archivo Excel: [Errno 2] No such file or directory: '/content/sdg_05_50_spreadsheet.xlsx'


NameError: name 'df' is not defined

In [ ]:
# Realizamos una copia del dataset y calculamos gap
df_m = df.copy()
df_m['Gap'] = 100 - 2*df_m['Porcentaje']
# Realizamos copia para asignar al género hombre
df_h = df_m.copy()
# Agregamos la columna Género, así podemos armar un Ranged Dot Plot
df_m['Género'] = 'Mujer'
df_h['Género'] = 'Hombre'
# Establecemos el porcentaje correspondiente a los hombres
df_h['Porcentaje'] = 100 - df_m['Porcentaje']
# Concatenamos ambos subsets
df_2 = pd.concat([df_m, df_h], axis=0).reset_index(drop=True)
display(df_2.sort_values('Gap'))

In [ ]:
import altair as alt
from vega_datasets import data

# Seleccionamos los valores correspondientes a EU, países individuales de la EU
source = df_2[(df_2['Grupo'] == 'EU') & ~(df_2['Region'].str.contains('28'))]
# Valores que nos serań útiles
min_yr = df_2['Año'].min()
max_yr = df_2['Año'].max()

# Condiciones de formato:
formato = lambda cond, true, false: (cond, alt.value(true), alt.value(false))

# Armamos un slider que nos permita seleccionar dinámicamente el año
slider = alt.binding_range(min=min_yr, max=max_yr, step=1, name="Año  ")
select_year = alt.selection_point(name='Año', fields=['Año'],
                                   bind=slider, value=max_yr)
# Efecto sobre la visualización para resaltar el punto de interés
highlight = alt.selection_point(
    on="pointerover", fields=["Entidad"], nearest=True
)

# Armamos la base de nuestro chart
chart = alt.Chart(
    data=source,
    title=alt.Title(
       "Brecha de género en Parlamentos de la UE",
       subtitle="Proporción de representantes en Parlamentos de la UE por género y país"
   )
).transform_filter(
    filter={'field': 'Género',
            "oneOf": ['Mujer', 'Hombre']}
).add_params(
    select_year
).transform_filter(
    select_year
).transform_filter(
    'isValid(datum.Porcentaje)'
)

# Definimos las líneas que representan el gap entre los valores de mujeres y varones
gap = chart.mark_line().encode(
    x=alt.X('Porcentaje:Q', # Fijamos la escala para que se mantenga constante independiente de los valores anuales
            scale=alt.Scale(domain=[0, 100]), ),
    y=alt.Y('Entidad:N', title="",
            axis=alt.Axis( # Formateamos los ejes dependiendo del valor
                          labelColor=alt.condition(*formato('datum.value == "EU"', '#4577A0','black')),
                          labelFontWeight=900
                )
            ).sort(
                field='Gap',
                op='sum',
                order='ascending'
    ),
    color=alt.condition(*formato('datum.Entidad == "EU"', '#4577A0','#5E666E')),
    detail='Region:N',
    tooltip=['Entidad:N', 'Año:N', 'Gap:Q'],
    strokeWidth=alt.condition(*formato(~highlight, 5, 10)),
    strokeOpacity=alt.condition(*formato(~highlight, 0.5, 1))
)

# Definimos los puntos de los porcentajes de representación por género
points = chart.mark_point(
    filled=True
).encode(
    x=alt.X('Porcentaje:Q', title="Seats proportion by gender (%)",
            # Fijamos la escala para que se mantenga constante independiente de los valores anuales
            scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('Entidad:N', title="",
            ).sort( # Ordenamos por tamaño del gap (ascendiente)
        field='Gap',
        op='sum',
        order='ascending'
    ),
    color=alt.Color('Género:O', legend=alt.Legend(
        orient='none', titleOrient='left',
        legendX=80, legendY=-20,
        direction='horizontal',
        titleAnchor='middle')
    ).scale(
        domain=['Mujer', 'Hombre'],
        range=['#0D6D64', '#F4BA3B']), # Utilizamos un color gender-neutral
    size=alt.condition(*formato(~highlight, 100, 500)),
    opacity=alt.condition(*formato(~highlight, 0.75, 1)),
    tooltip=[ 'Género:N', 'Entidad:N', 'Porcentaje:Q', 'Año:N']
    #text=alt.condition(*formato(~highlight, 'Porcentaje:Q', '  ')),
).add_params(
    highlight
).interactive()



(gap + points)

In [ ]:
# Otra prueba

color_scheme = 'set3'
color = pd.read_table(
    'https://raw.githubusercontent.com/vega/vega/v5.21.0/packages/vega-scale/src/palettes.js',
    skipinitialspace=True,
    sep=':',
).loc[
    color_scheme
].str.replace(
    "'",
    ""
).str.replace(
    ",",
    ""
).apply(
    lambda x: ["#" + x[i:i+6] for i in range(0, len(x), 6)]
)

col = color[0]


source2 = df_2.copy()

# Valores que nos serań útiles
min_yr = df_2['Año'].min()
max_yr = df_2['Año'].max()

# Condiciones de formato:
formato = lambda cond, true, false: (cond, alt.value(true), alt.value(false))

# Armamos un slider que nos permita seleccionar dinámicamente el año
slider = alt.binding_range(min=min_yr, max=max_yr, step=1, name="Año  ")
select_year = alt.selection_point(name='Año', fields=['Año'],
                                   bind=slider, value=max_yr)
# Efecto sobre la visualización para resaltar el punto de interés
highlight = alt.selection_point(
    on="pointerover", fields=["Entidad"], nearest=True
)


#color = alt.Color('Entidad:N').scale()

# We create two selections:
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
brush = alt.selection_interval(encodings=['y'])
click = alt.selection_point(encodings=['y'])


chart = alt.Chart(
    data=source2,
    #title=alt.Title(
    #   "Proporción de representantes en Parlamentos de la UE por género y país"
       #subtitle=
  # )
).transform_filter(
    filter={'field': 'Género',
            "oneOf": ['Mujer', 'Hombre']}
).add_params(
    select_year
).transform_filter(
    select_year
).transform_filter(
    'isValid(datum.Porcentaje)'
)

# Definimos las líneas que representan el gap entre los valores de mujeres y varones
gap = chart.mark_line().encode(
    x=alt.X('Porcentaje:Q', # Fijamos la escala para que se mantenga constante independiente de los valores anuales
            scale=alt.Scale(domain=[0, 100]), ),
    y=alt.Y('Entidad:N', title="",
            axis=alt.Axis( # Formateamos los ejes dependiendo del valor
                          labelColor=alt.condition(*formato('datum.value == "EU"', '#4577A0','black')),
                          labelFontWeight=900
                )
            ).sort(
                field='Gap',
                op='sum',
                order='ascending'
    ),
    color=alt.condition(*formato('datum.Entidad == "EU"', '#4577A0','#5E666E')),
    detail='Region:N',
    strokeWidth=alt.condition(*formato(~highlight, 5, 10)),
    strokeOpacity=alt.condition(*formato(~highlight, 0.5, 1))
)


# Definimos los puntos de los porcentajes de representación por género
points = chart.mark_point(
    filled=True
).encode(
    x=alt.X('Porcentaje:Q', title="Seats proportion by gender (%)",
            # Fijamos la escala para que se mantenga constante independiente de los valores anuales
            scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('Entidad:N', title="",
            ).sort( # Ordenamos por tamaño del gap (ascendiente)
        field='Gap',
        op='sum',
        order='ascending'
    ),
    color=alt.Color('Género:O', legend=alt.Legend(
        orient='none', titleOrient='left',
        legendX=150, legendY=-20,
        direction='horizontal',
        titleAnchor='middle')
    ).scale(
        domain=['Mujer', 'Hombre'],
        range=['#0D6D64', '#F4BA3B']), # Utilizamos un color gender-neutral
    size=alt.condition(*formato(~highlight, 100, 500)),
    opacity=alt.condition(*formato(~highlight, 0.75, 1)),
    tooltip=[ 'Género:N', 'Entidad:N', 'Porcentaje:Q', 'Año:N']
).properties(
    width=550,
    height=800
).add_params(
    brush
).add_params(
    highlight
).transform_filter(
    click
)

# Definimos las líneas que representan % de un país seleccionado over time
bars = alt.Chart(source2).mark_line(
    point=alt.OverlayMarkDef(filled=False, fill="white")
    ).encode(
    x='Año:N',
    y='Porcentaje:Q',
    color=alt.condition(~highlight, alt.value('red'), alt.value('lightgray')),
).transform_filter(
    brush
).transform_filter(
     alt.FieldEqualPredicate(field='Género', equal='Mujer')
).properties(
    width=550,
    height=800
).add_params(
    click
).interactive()



alt.hconcat(
    (gap + points),
    bars,
    title="Brecha de género en Parlamentos de la UE"
)

In [ ]:
color_scheme = 'set3'
color = pd.read_table(
    'https://raw.githubusercontent.com/vega/vega/v5.21.0/packages/vega-scale/src/palettes.js',
    skipinitialspace=True,
    sep=':',
).loc[
    color_scheme
].str.replace(
    "'",
    ""
).str.replace(
    ",",
    ""
).apply(
    lambda x: ["#" + x[i:i+6] for i in range(0, len(x), 6)]
)

color[0]